In [1]:
import time
import pytz
import pandas as pd
import base58
import datetime
from solana.rpc.api import Client
from solders.pubkey import Pubkey

In [2]:
class TransactionLine:
    def __init__(self, pubkey, tx, tx_detail):
        self.pubkey = pubkey
        self.tx = tx
        self.tx_detail = tx_detail
    
    @property
    def time(self):
        dt_object = datetime.datetime.fromtimestamp(self.tx.block_time, pytz.utc)
        pst_timezone = pytz.timezone('US/Pacific')
        pst_time = dt_object.astimezone(pst_timezone)
        return pst_time.strftime('%Y-%m-%d %I:%M:%S %p')
    
    @property
    def _pre_balances(self):
        return [bal / 1e9 for bal in self.tx_detail.value.transaction.meta.pre_balances]

    @property
    def _post_balances(self):
        return [bal / 1e9 for bal in self.tx_detail.value.transaction.meta.post_balances]

    @property
    def _account_keys(self):
        return [key.__str__() for key in tx_details.value.transaction.transaction.message.account_keys]
    
    @property
    def _self_index(self):
        for i, key in enumerate(self._account_keys):
            if key == self.pubkey:
                return i
    
    @property
    def balance(self):
        return self._post_balances[self._self_index]

    @property
    def amount(self):
        index = self._self_index
        return self._post_balances[index] - self._pre_balances[index]

    @property
    def to_dict(self):
        return {
            'time': self.time,
            'amount': self.amount,
            'balance': self.balance,
        }
        

In [8]:
# client = Client("https://api.mainnet-beta.solana.com")
client = Client("http://127.0.0.1:8899")

wallet_address = "NLMSHTjmSiRxGJPs3uaqtsFBC2dTGYwK41U18Nmw5kH"

def get_transactions(wallet_address):
    return client.get_signatures_for_address(Pubkey(wallet_address))

def calculate_total_value(transactions):
    total_value = 0
    for tx in transactions:
        tx_details = client.get_transaction(tx.signature)
        for instruction in tx_details['result']['transaction']['message']['instructions']:
            if 'parsed' in instruction:
                if instruction['parsed']['type'] == 'transfer':
                    total_value += int(instruction['parsed']['info']['lamports'])
    return total_value

In [10]:
decoded_address = base58.b58decode(wallet_address)
transactions = get_transactions(decoded_address)

In [16]:
lines = []
for i, transaction in enumerate(transactions.value):
    if i > 500:
        break
    # time.sleep(5) # temp sleep since we're using public rpc
    tx_details = client.get_transaction(transaction.signature)
    lines.append(TransactionLine(wallet_address, transaction, tx_details).to_dict)

In [17]:
pd.DataFrame(lines)

,time,amount,balance
0,2025-01-23 11:35:39 PM,-0.000005,7.271170
1,2025-01-23 11:35:38 PM,-0.000005,7.271175
2,2025-01-23 11:35:38 PM,-0.000005,7.271180
3,2025-01-23 11:35:37 PM,-0.000005,7.271185
4,2025-01-23 11:35:37 PM,-0.000005,7.271190
...,...,...,...
496,2025-01-23 11:32:17 PM,-0.000005,7.273650
497,2025-01-23 11:32:17 PM,-0.000005,7.273655
498,2025-01-23 11:32:17 PM,-0.000005,7.273660
499,2025-01-23 11:32:16 PM,-0.000005,7.273665
